In [8]:
# ==============================================================================
# WILDFIRE EVACUATION THREAT PREDICTION -- LightGBM Survival Model
# Goal: Hybrid Score >= 0.99 (0.3 * C-index + 0.7 * (1 - Weighted Brier Score))
# ==============================================================================

# -- STEP 0: Install dependencies ----------------------------------------------
# !pip install lightgbm scikit-survival optuna shap lifelines --quiet

import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
import shap
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

SEED = 42
np.random.seed(SEED)


In [9]:
# ==============================================================================
# STEP 1: Load Data
# ==============================================================================

train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
sub   = pd.read_csv('sample_submission.csv')

print(f"Train: {train.shape} | Test: {test.shape}")
print(f"Event rate (fires that hit): {train['event'].mean():.1%}")
print(f"Time-to-hit range: {train['time_to_hit_hours'].min():.1f}h to {train['time_to_hit_hours'].max():.1f}h")

Train: (221, 37) | Test: (95, 35)
Event rate (fires that hit): 31.2%
Time-to-hit range: 0.0h to 67.0h


In [10]:
# ==============================================================================
# STEP 2: Feature Engineering
# ==============================================================================

def engineer_features(df):
    df = df.copy()

    # Proximity danger score -- close + fast growing = biggest threat
    df['danger_score'] = df['area_growth_rate_ha_per_h'] / (df['dist_min_ci_0_5h'] + 1)

    # Naive time-to-close-the-gap estimate (distance / closing speed)
    # Gives the model a direct survival-time signal
    safe_closing = df['closing_speed_m_per_h'].clip(lower=1)
    df['eta_hours_naive'] = (df['dist_min_ci_0_5h'] / safe_closing).clip(upper=72)

    # Binary risk flags for clean split boundaries
    df['is_fast_grower']   = (df['area_growth_rate_ha_per_h'] > df['area_growth_rate_ha_per_h'].quantile(0.75)).astype(int)
    df['is_close_to_evac'] = (df['dist_min_ci_0_5h'] < df['dist_min_ci_0_5h'].quantile(0.25)).astype(int)
    df['is_low_res']       = df['low_temporal_resolution_0_5h'].astype(int)

    # Spread geometry
    df['is_advancing']   = (df['along_track_speed'] > 0).astype(int)
    df['aligned_threat'] = df['alignment_abs'] * df['closing_speed_abs_m_per_h']

    # Log-transform skewed features
    df['log_dist_min']      = np.log1p(df['dist_min_ci_0_5h'])
    df['log_closing_speed'] = np.log1p(df['closing_speed_abs_m_per_h'])
    df['log_eta']           = np.log1p(df['eta_hours_naive'])

    # Negative acceleration = fire accelerating toward evac zone
    df['is_accelerating_toward'] = (df['dist_accel_m_per_h2'] < 0).astype(int)

    # Interaction: large fire heading toward a zone is a compounding risk
    df['area_x_alignment'] = df['log1p_area_first'] * df['alignment_abs']

    # Temporal context: afternoon ignitions and fire season months are more dangerous
    df['is_peak_hour']  = ((df['event_start_hour'] >= 12) & (df['event_start_hour'] <= 18)).astype(int)
    df['is_peak_month'] = df['event_start_month'].isin([6, 7, 8, 9]).astype(int)

    return df

train = engineer_features(train)
test  = engineer_features(test)

NON_FEATURES = ['event_id', 'time_to_hit_hours', 'event']
FEATURE_COLS = [c for c in train.columns if c not in NON_FEATURES]
print(f"\nTotal features after engineering: {len(FEATURE_COLS)}")



Total features after engineering: 48


In [11]:
# ==============================================================================
# STEP 3: Build Binary Targets for Each Horizon
# ==============================================================================
# For each horizon H:
#   POSITIVE:  event=1 AND time_to_hit <= H  (confirmed hit before horizon)
#   NEGATIVE:  time_to_hit > H               (fire survived past horizon)
#   EXCLUDED:  event=0 AND time_to_hit <= H  (censored before horizon -- truth unknown)

HORIZONS = [12, 24, 48, 72]

def make_binary_targets(df):
    masks = {}
    for H in HORIZONS:
        hit     = (df['event'] == 1) & (df['time_to_hit_hours'] <= H)
        no_hit  = df['time_to_hit_hours'] > H
        include = hit | no_hit
        df[f'hit_{H}h'] = hit.astype(int)
        masks[f'mask_{H}h'] = include
    return df, masks

train, horizon_masks = make_binary_targets(train)

for H in HORIZONS:
    mask     = horizon_masks[f'mask_{H}h']
    hit_rate = train.loc[mask, f'hit_{H}h'].mean()
    print(f"Horizon {H:2d}h -> usable samples: {mask.sum():3d} | hit rate: {hit_rate:.1%}")

Horizon 12h -> usable samples: 215 | hit rate: 22.8%
Horizon 24h -> usable samples: 196 | hit rate: 32.1%
Horizon 48h -> usable samples: 166 | hit rate: 39.8%
Horizon 72h -> usable samples:  69 | hit rate: 100.0%


In [12]:
# ==============================================================================
# STEP 4: Optuna Hyperparameter Tuning
# ==============================================================================
# Tune on the 48h horizon (40% weight in scoring) and reuse params for all horizons.

mask_48 = horizon_masks['mask_48h']
X_48    = train.loc[mask_48, FEATURE_COLS].reset_index(drop=True)
y_48    = train.loc[mask_48, 'hit_48h'].reset_index(drop=True)

def lgb_cv_score(trial):
    params = {
        'objective':         'binary',
        'metric':            'binary_logloss',
        'verbosity':         -1,
        'n_jobs':            -1,
        'seed':              SEED,
        'num_leaves':        trial.suggest_int('num_leaves', 8, 64),
        'learning_rate':     trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'max_depth':         trial.suggest_int('max_depth', 3, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 40),
        'subsample':         trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha':         trial.suggest_float('reg_alpha', 1e-4, 5.0, log=True),
        'reg_lambda':        trial.suggest_float('reg_lambda', 1e-4, 5.0, log=True),
        'n_estimators':      trial.suggest_int('n_estimators', 100, 600),
    }
    skf    = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []
    for tr_idx, va_idx in skf.split(X_48, y_48):
        model       = lgb.LGBMClassifier(**params)
        val_classes = np.unique(y_48.iloc[va_idx])
        if len(val_classes) < 2:
            model.fit(X_48.iloc[tr_idx], y_48.iloc[tr_idx],
                      callbacks=[lgb.log_evaluation(-1)])
            scores.append(0.0)
        else:
            model.fit(X_48.iloc[tr_idx], y_48.iloc[tr_idx],
                      eval_set=[(X_48.iloc[va_idx], y_48.iloc[va_idx])],
                      callbacks=[lgb.early_stopping(30, verbose=False),
                                 lgb.log_evaluation(-1)])
            preds = model.predict_proba(X_48.iloc[va_idx])[:, 1]
            scores.append(log_loss(y_48.iloc[va_idx], preds))
    return np.mean(scores)

print("\nRunning Optuna hyperparameter search (50 trials)...")
study = optuna.create_study(direction='minimize',
                            sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(lgb_cv_score, n_trials=50, show_progress_bar=True)

BEST_PARAMS = study.best_params
BEST_PARAMS.update({'objective': 'binary', 'metric': 'binary_logloss',
                    'verbosity': -1, 'n_jobs': -1, 'seed': SEED})
print(f"\nBest params: {BEST_PARAMS}")
print(f"Best CV log-loss: {study.best_value:.4f}")



Running Optuna hyperparameter search (50 trials)...


  0%|          | 0/50 [00:00<?, ?it/s]


Best params: {'num_leaves': 26, 'learning_rate': 0.07347677222383225, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.8764598226325977, 'colsample_bytree': 0.5934558555856326, 'reg_alpha': 0.001034527831103876, 'reg_lambda': 0.0029559118884026404, 'n_estimators': 319, 'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'n_jobs': -1, 'seed': 42}
Best CV log-loss: 0.0601


In [13]:
# ==============================================================================
# STEP 5: Train One LightGBM per Horizon with Cross-Validation OOF Predictions
# ==============================================================================
# OOF predictions are stored as {event_id: prob} dicts to avoid pandas index
# alignment bugs -- a common silent failure when mixing boolean masks + numpy arrays.

N_FOLDS     = 5
test_preds  = pd.DataFrame({'event_id': test['event_id']})
X_test      = test[FEATURE_COLS]
oof_records = {H: {} for H in HORIZONS}

for H in HORIZONS:
    col  = f'hit_{H}h'
    mask = horizon_masks[f'mask_{H}h']

    # Reset index so positional iloc and numpy arrays stay aligned
    X_h   = train.loc[mask, FEATURE_COLS].reset_index(drop=True)
    y_h   = train.loc[mask, col].reset_index(drop=True)
    ids_h = train.loc[mask, 'event_id'].reset_index(drop=True)

    skf             = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    oof             = np.zeros(len(X_h))
    test_fold_preds = np.zeros(len(X_test))

    print(f"\n-- Horizon {H}h -- ({mask.sum()} samples, {y_h.mean():.1%} hit rate)")

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_h, y_h)):
        model       = lgb.LGBMClassifier(**BEST_PARAMS)
        val_classes = np.unique(y_h.iloc[va_idx])

        if len(val_classes) < 2:
            # Single-class val fold (common at 72h with 100% hit rate).
            # binary_logloss eval would crash, so train without eval_set.
            model.fit(X_h.iloc[tr_idx], y_h.iloc[tr_idx],
                      callbacks=[lgb.log_evaluation(-1)])
            print(f"  Fold {fold+1}: skipped early stopping (only class {val_classes} in val)")
        else:
            model.fit(X_h.iloc[tr_idx], y_h.iloc[tr_idx],
                      eval_set=[(X_h.iloc[va_idx], y_h.iloc[va_idx])],
                      callbacks=[lgb.early_stopping(50, verbose=False),
                                 lgb.log_evaluation(-1)])
            fold_preds = model.predict_proba(X_h.iloc[va_idx])[:, 1]
            print(f"  Fold {fold+1}: log-loss = {log_loss(y_h.iloc[va_idx], fold_preds):.4f}")

        oof[va_idx]      = model.predict_proba(X_h.iloc[va_idx])[:, 1]
        test_fold_preds += model.predict_proba(X_test)[:, 1] / N_FOLDS

    for i, eid in enumerate(ids_h):
        oof_records[H][eid] = oof[i]

    test_preds[f'prob_{H}h'] = test_fold_preds

# Reconstruct oof_preds via safe event_id mapping
oof_preds = pd.DataFrame({'event_id': train['event_id']})
for H in HORIZONS:
    oof_preds[f'prob_{H}h'] = oof_preds['event_id'].map(oof_records[H])

print("\nAll horizons trained!")



-- Horizon 12h -- (215 samples, 22.8% hit rate)
  Fold 1: log-loss = 0.2399
  Fold 2: log-loss = 0.1456
  Fold 3: log-loss = 0.0556
  Fold 4: log-loss = 0.1355
  Fold 5: log-loss = 0.2483

-- Horizon 24h -- (196 samples, 32.1% hit rate)
  Fold 1: log-loss = 0.0204
  Fold 2: log-loss = 0.0984
  Fold 3: log-loss = 0.0857
  Fold 4: log-loss = 0.1050
  Fold 5: log-loss = 0.1626

-- Horizon 48h -- (166 samples, 39.8% hit rate)
  Fold 1: log-loss = 0.0560
  Fold 2: log-loss = 0.1478
  Fold 3: log-loss = 0.0209
  Fold 4: log-loss = 0.0754
  Fold 5: log-loss = 0.0003

-- Horizon 72h -- (69 samples, 100.0% hit rate)
  Fold 1: skipped early stopping (only class [1] in val)
  Fold 2: skipped early stopping (only class [1] in val)
  Fold 3: skipped early stopping (only class [1] in val)
  Fold 4: skipped early stopping (only class [1] in val)
  Fold 5: skipped early stopping (only class [1] in val)

All horizons trained!


In [14]:
# ==============================================================================
# STEP 6: Enforce Monotonicity
# ==============================================================================

def enforce_monotonicity(df):
    df = df.copy()
    df['prob_24h'] = df[['prob_12h', 'prob_24h']].max(axis=1)
    df['prob_48h'] = df[['prob_24h', 'prob_48h']].max(axis=1)
    df['prob_72h'] = df[['prob_48h', 'prob_72h']].max(axis=1)
    return df

test_preds = enforce_monotonicity(test_preds)

prob_cols      = [f'prob_{H}h' for H in HORIZONS]
has_any_pred   = oof_preds[prob_cols].notna().any(axis=1)
oof_preds_eval = enforce_monotonicity(oof_preds[has_any_pred].copy())
print(f"OOF rows available for evaluation: {has_any_pred.sum()}")

OOF rows available for evaluation: 215


In [15]:
# ==============================================================================
# STEP 7: Evaluate OOF Predictions
# ==============================================================================

from sklearn.metrics import brier_score_loss

def compute_brier_censored(event, time_to_hit, y_pred, horizon):
    include = ((event == 1) & (time_to_hit <= horizon)) | (time_to_hit > horizon)
    label   = ((event == 1) & (time_to_hit <= horizon)).astype(int)
    if include.sum() == 0:
        return np.nan
    return brier_score_loss(label[include], y_pred[include])

oof_eval = oof_preds_eval.merge(
    train[['event_id', 'event', 'time_to_hit_hours']], on='event_id', how='left'
)

briers = {}
for H in [24, 48, 72]:
    col   = f'prob_{H}h'
    valid = oof_eval[col].notna()
    bs = compute_brier_censored(
        oof_eval.loc[valid, 'event'].values,
        oof_eval.loc[valid, 'time_to_hit_hours'].values,
        oof_eval.loc[valid, col].values,
        H
    )
    briers[H] = bs
    print(f"OOF Brier@{H}h: {bs:.4f}")

weighted_brier = 0.3 * briers[24] + 0.4 * briers[48] + 0.3 * briers[72]
print(f"\nWeighted Brier Score: {weighted_brier:.4f}")
print(f"(1 - Weighted Brier): {1 - weighted_brier:.4f}")

try:
    from lifelines.utils import concordance_index
    c_idx  = concordance_index(
        oof_eval['time_to_hit_hours'],
        -oof_eval['prob_72h'],
        oof_eval['event']
    )
    hybrid = 0.3 * c_idx + 0.7 * (1 - weighted_brier)
    print(f"C-index (OOF):          {c_idx:.4f}")
    print(f"Estimated Hybrid Score: {hybrid:.4f}")
except ImportError:
    print("Install lifelines for C-index: !pip install lifelines")

OOF Brier@24h: 0.0244
OOF Brier@48h: 0.0175
OOF Brier@72h: 0.0052

Weighted Brier Score: 0.0159
(1 - Weighted Brier): 0.9841
Install lifelines for C-index: !pip install lifelines


In [16]:
# ==============================================================================
# STEP 8: SHAP Feature Importance
# ==============================================================================

print("\nComputing SHAP values for 48h horizon model...")
mask_48        = horizon_masks['mask_48h']
final_model_48 = lgb.LGBMClassifier(**BEST_PARAMS)
final_model_48.fit(train.loc[mask_48, FEATURE_COLS], train.loc[mask_48, 'hit_48h'])

explainer = shap.TreeExplainer(final_model_48)
shap_vals = explainer.shap_values(train.loc[mask_48, FEATURE_COLS])

if isinstance(shap_vals, list):
    shap_vals = shap_vals[1]

shap_importance = pd.DataFrame({
    'feature':   FEATURE_COLS,
    'mean_shap': np.abs(shap_vals).mean(axis=0)
}).sort_values('mean_shap', ascending=False)

print("\nTop 15 most important features (by mean |SHAP|):")
print(shap_importance.head(15).to_string(index=False))


Computing SHAP values for 48h horizon model...

Top 15 most important features (by mean |SHAP|):
                     feature  mean_shap
            dist_min_ci_0_5h   6.205617
                log_dist_min   3.080733
            event_start_hour   0.826264
         num_perimeters_0_5h   0.654496
            is_close_to_evac   0.510823
               area_first_ha   0.469008
               alignment_abs   0.367906
           event_start_month   0.272524
            log1p_area_first   0.189738
          dist_slope_ci_0_5h   0.143965
       event_start_dayofweek   0.143117
          dt_first_last_0_5h   0.142685
            area_x_alignment   0.055772
                is_peak_hour   0.052506
low_temporal_resolution_0_5h   0.031037


In [17]:
# ==============================================================================
# STEP 9: Calibration Pass (Platt Scaling)
# ==============================================================================
# Fits a logistic regression on OOF predictions vs true labels, then applies
# that correction to test predictions. Directly reduces Brier score.

calibrated_test_preds = test_preds.copy()

for H in HORIZONS:
    col = f'prob_{H}h'

    oof_h  = oof_preds[['event_id', col]].dropna(subset=[col])
    cal_df = oof_h.merge(
        train[['event_id', f'hit_{H}h']], on='event_id', how='inner'
    )

    if len(cal_df) < 10:
        print(f"  {H}h: not enough OOF samples to calibrate, skipping")
        continue

    oof_sub = cal_df[col].values
    y_sub   = cal_df[f'hit_{H}h'].values

    if len(np.unique(y_sub)) < 2:
        # 72h is 100% hits -- no negative class to fit against.
        # Cap slightly below 1.0 to avoid max Brier penalty on any wrong prediction.
        print(f"  {H}h: only one class in OOF labels -- capping test preds at 0.97")
        calibrated_test_preds[col] = calibrated_test_preds[col].clip(upper=0.97)
        continue

    platt = LogisticRegression(C=1.0, solver='lbfgs')
    platt.fit(oof_sub.reshape(-1, 1), y_sub)
    calibrated_test_preds[col] = platt.predict_proba(
        test_preds[col].values.reshape(-1, 1)
    )[:, 1]
    print(f"  Calibrated {H}h: test mean prob {calibrated_test_preds[col].mean():.3f}")

calibrated_test_preds = enforce_monotonicity(calibrated_test_preds)

  Calibrated 12h: test mean prob 0.178
  Calibrated 24h: test mean prob 0.273
  Calibrated 48h: test mean prob 0.292
  72h: only one class in OOF labels -- capping test preds at 0.97


In [18]:
# ==============================================================================
# STEP 10: Build and Validate Submission
# ==============================================================================

submission     = calibrated_test_preds[['event_id', 'prob_12h', 'prob_24h', 'prob_48h', 'prob_72h']].copy()
prob_col_names = ['prob_12h', 'prob_24h', 'prob_48h', 'prob_72h']

assert submission['event_id'].nunique() == len(submission), \
    "Duplicate IDs found!"
assert set(submission['event_id']) == set(sub['event_id']), \
    "ID mismatch with sample submission!"

# DataFrame does not have .between() -- apply it column-wise instead
in_range = submission[prob_col_names].apply(lambda col: col.between(0, 1)).all().all()
assert in_range, "Probabilities out of [0, 1]!"

violations = (
    (submission['prob_12h'] > submission['prob_24h']) |
    (submission['prob_24h'] > submission['prob_48h']) |
    (submission['prob_48h'] > submission['prob_72h'])
)
assert not violations.any(), f"Monotonicity violated in {violations.sum()} rows!"

print("\nAll submission checks passed!")
print(submission[prob_col_names].describe().round(3))

submission.to_csv('submission.csv', index=False)
print("\nSaved: submission.csv")

# ==============================================================================
# BONUS: What to try if score is not high enough
# ==============================================================================
# 1. Increase Optuna trials (n_trials=100+)
# 2. Blend with RandomSurvivalForest from scikit-survival for better C-index ranking
# 3. Stack: train a meta-model on OOF probs from multiple base model types
# 4. Try isotonic regression calibration as an alternative to Platt scaling
# 5. Engineer fire-behavior features: wind alignment, terrain slope if available


All submission checks passed!
       prob_12h  prob_24h  prob_48h  prob_72h
count    95.000    95.000    95.000    95.000
mean      0.178     0.274     0.292     0.314
std       0.226     0.353     0.370     0.403
min       0.052     0.052     0.055     0.055
25%       0.052     0.052     0.056     0.056
50%       0.053     0.053     0.057     0.057
75%       0.244     0.770     0.872     0.941
max       0.815     0.882     0.898     0.970

Saved: submission.csv
